In [96]:
import OpenOPC as OP
import threading as th
import pywintypes
import time
pywintypes.datetime=pywintypes.TimeType
import socket

### Definicion de variables globales

In [3]:
opcserver = "ABB.AC800MC_OpcDaSarver.3"

#lista de comandos:
#cerrar: finaliza el programa
close="5"
#iniciar: inicia la medicion
start= "1"
#detener: detiene la medicion
stop="2"
#Diccionario para mensajes luego de los comandos
comdic={start:"Interfaz iniciada", stop:"Interfaz detenida", close:"Cerrando programa"}
comando=stop
comprev=comando
variables=[]

Conecto con el servidor OPC

In [2]:
opc=OP.client()
opc.connect('ABB.AC800MC_OpcDaServer.3')

### Funciones


In [8]:
## Funcion que permite ingresar comandos desde consola
def comandos():
    "Funcion que permite ingresar comandos desde consola"
    global comando, close, comdic, comprev
    
    while comando!=close:
        comando1 =input("\ningrese comando:\n  1. Iniciar interfaz\n  2. Detener interfaz\n  5. Salir\n")
        if comando1 in comdic:
            comprev=comando
            comando=comando1
            print(comdic[comando])
        else:
            print("el comando {} no se reconoce".format(comando1))

In [93]:
#funcion que lee el OPC
def actualizar():
    vars=list(opc.read(["Applications.Application_1.b1","Applications.Application_1.Program2.int1","Applications.Application_1.Program2.bool1"]))
    vars2= list(map((lambda x: 1 if x[1]==True else (0 if x[1]==False else x[1])),vars))
    return vars2

In [94]:
actualizar()

[0, 1, 0]

listas=e[1:-1].split(",")
listas

### Rutina principal

In [8]:
##### Thread 2, Busca comandos
#th2=th.Thread(target=comandos,name="th2")
#th2.start()
#th1=th.Thread(target=comandos,name="th1")
host="192.168.99.10"
port=9999
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((host, port))
    ## Loop principal
    while True:
        s.listen()
        conn, addr = s.accept()
        with conn:
            print(f"Connected by {addr}")
            while True:
                data = conn.recv(1024).decode("ascii")
                if not data or data=="cerrar":
                    break
                print(data)
                resp=str(actualizar()).encode("ascii")
                #resp=responder(data) 
                #resp=b'25.75,5.45,12.152,456,456.2,1,3215,123,123,123,515,15,681,6,313,6,5558, abcde,int32,mi vieja mula ya no es lo que era'
                conn.sendall(resp)  

Interfaz iniciada
(1, True, 4)
(1, True, 4)
(1, True, 4)
Interfaz detenida
(1, True, 4)
Interfaz iniciada
(1, False, 5)
(1, False, 5)
Interfaz detenida
(1, True, 5)
Cerrando programa


ProgrammingError: Attempt to use a closed cursor.